<a href="https://colab.research.google.com/github/A01793509/Equipo38/blob/main/MNA_IAyAA_semana_10_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Alberto Jose Garcia Porras (A01793509)
*   Carlos Julio León Caicedo (A01793947)
*   Luis Fernando Ríos Piedra (A00453954)
*   Marco Antonio Vázquez Morales (A01793704)


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [19]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, zipfile
import seaborn as sns
from io import BytesIO

from sklearn.decomposition import TruncatedSVD,randomized_svd

#**Ejercicio-1.** 

1. Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones. 

In [42]:
#Extracción de rating_final.csv y geoplaces2.csv desde el sitio fuente

path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
base_datos = path.split('/')[-1]
req_path = requests.get(path)

zipfile.ZipFile(BytesIO(req_path.content)).extractall('/content/sample_data/RCdata')
df_rt = pd.read_csv("/content/sample_data/RCdata/rating_final.csv",sep=',')

path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
base_datos = path.split('/')[-1]
req_path = requests.get(path)

zipfile.ZipFile(BytesIO(req_path.content)).extractall('/content/sample_data/RCdata')
df_g = pd.read_csv("/content/sample_data/RCdata/geoplaces2.csv",sep=',',encoding='latin-1')


In [45]:
#Generar matriz relacional y asignar index con userID
df_rel=pd.merge(df_rt,df_g,how='inner')
df_rel=df_rel.set_index('userID')


In [48]:
df_rel.shape

(1161, 24)

In [53]:
#Generar matriz de utilidad e impresión de 10 renglones
UtMx = df_rel.pivot_table(values='rating', index='userID', columns='name', fill_value=0)

UtMx.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [54]:
#Desplegar tamaño de la matriz
#Tenemos 138 usuarios y 129 restaurantes
UtMx.shape 

(138, 129)

#**Ejercicio-2.**

In [17]:
print('Total de elementos de la matriz de utilidad: %d' % (UtMx.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtMx)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtMx) / UtMx.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Porcentaje de elementos diferentes de cero: 5.1%


In [55]:
X = UtMx.T
X.shape

(129, 138)

In [56]:


SVD = TruncatedSVD(n_components=137, random_state=1)  
SVD.fit(X)

print('La cantidad total de valores singulares es de: %d' % (SVD.singular_values_).shape[0])


La cantidad total de valores singulares es de: 129


#**Ejercicio-3.**

#**Ejercicio-4.**

In [ ]:
None

#**Ejercicio-5.**

In [ ]:
None

#**Ejercicio-6.**

In [ ]:
None

#**Ejercicio-7.**

In [ ]:
None

#**Ejercicio-8.**

In [ ]:
None

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**